In [1]:
!pip install pandas
!pip install pyspark
!pip install pyarrow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

24/10/28 21:20:06 WARN Utils: Your hostname, Maazs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.2.24 instead (on interface en0)
24/10/28 21:20:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/28 21:20:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})

In [4]:
signups = spark.createDataFrame(signups)
signups.show()

confirmations = spark.createDataFrame(confirmations)
confirmations.show()

+-------+-------------------+
|user_id|         time_stamp|
+-------+-------------------+
|      3|2020-03-21 10:16:13|
|      7|2020-01-04 13:57:59|
|      2|2020-07-29 23:09:44|
|      6|2020-12-09 10:39:37|
+-------+-------------------+

+-------+-------------------+---------+
|user_id|         time_stamp|   action|
+-------+-------------------+---------+
|      3|2021-01-06 03:30:46|  timeout|
|      3|2021-07-14 14:00:00|  timeout|
|      7|2021-06-12 11:57:29|confirmed|
|      7|2021-06-13 12:58:28|confirmed|
|      7|2021-06-14 13:59:27|confirmed|
|      2|2021-01-22 00:00:00|confirmed|
|      2|2021-02-28 23:59:59|  timeout|
+-------+-------------------+---------+



In [15]:
from pyspark.sql.functions import when, col, count, sum, round

signups \
    .join(
        confirmations,
        'user_id',
        'left'
    ) \
.withColumn('action', when(col('action') == "confirmed", 1).when(col('action') == "timeout", 0)) \
.fillna(0) \
.groupby('user_id') \
.agg(count('action').alias('total_requested'), 
     sum('action').alias('total_confirmed')) \
.withColumn('confirmation_rate', round(col('total_confirmed') / col('total_requested'), 2)) \
.select(['user_id', 'confirmation_rate']) \
.show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+

